## Spark SQL 튜토리얼
+ 데이터프레임 생성하기
+ SQL 처리를 위한 Temporary View 생성
+ 기본 SQL문 실습

In [6]:
# [+] SparkSession 임포트
from pyspark.sql import SparkSession

In [8]:
# [+] SparkSession 설정하기
ss = SparkSession.builder.master('local').appName('learn-sql').getOrCreate()

### 샘플 데이터셋: 주식 데이터
+ 변수: 회사명(name), 종목코드(ticker), 상장국가(country), 주가(price), 통화(currency)
+ 주가: Double 형식과 Long 형식이 섞이면 TypeError 발생하므로 Double(실수) 형태로 통일
    + 예: 삼성 주가, 67300 -> 67300.0


In [10]:
# 샘플 데이터셋

stocks = [
    ('Alphabet', 'GOOGL', 'USA', 2282.19, 'USD'), 
    ('Netflix', 'NFLX', 'USA', 190.36, 'USD'),
    ('Amazon', 'AMZN', 'USA', 2485.63, 'USD'),
    ('Tesla', 'TSLA', 'USA', 870.76, 'USD'),
    ('HUYA', 'HUYA', 'USA', 4.13, 'USD'),
    ('Tencent', '0700', 'Hong Kong', 377.4, 'HKD'),
    ('Toyota', '7203', 'Japan', 2228.0, 'JPY'),
    ('Samsung', '005930', 'Korea', 67300.0, 'KRW'),
    ('Kakao', '035720', 'Korea', 87300.0, 'KRW'),
    ('Com2uS', '078340', 'Korea', 95000.0, 'KRW')
]
# 한국은 소수점 X 주의!

In [11]:
# [+] 스키마 정의: 데이터 타입 자동으로 추론
stocksSchema = ['name', 'ticker', 'country', 'price', 'currency'] # 변수명 정의

In [14]:
# [+] 데이터프레임 생성
df = ss.createDataFrame(data=stocks, schema=stocksSchema) # schema정의 안해도 되지만 안하면 의미없음

In [16]:
# [+] 생성된 데이터프레임 확인
df.printSchema() # nullable : null값 허용여부(true는 허용)

root
 |-- name: string (nullable = true)
 |-- ticker: string (nullable = true)
 |-- country: string (nullable = true)
 |-- price: double (nullable = true)
 |-- currency: string (nullable = true)



In [18]:
# [+] 데이터프레임 출력
df.show() # RDD의 collect와 비슷한 역할. 데이터 보여줌

+--------+------+---------+-------+--------+
|    name|ticker|  country|  price|currency|
+--------+------+---------+-------+--------+
|Alphabet| GOOGL|      USA|2282.19|     USD|
| Netflix|  NFLX|      USA| 190.36|     USD|
|  Amazon|  AMZN|      USA|2485.63|     USD|
|   Tesla|  TSLA|      USA| 870.76|     USD|
|    HUYA|  HUYA|      USA|   4.13|     USD|
| Tencent|  0700|Hong Kong|  377.4|     HKD|
|  Toyota|  7203|    Japan| 2228.0|     JPY|
| Samsung|005930|    Korea|67300.0|     KRW|
|   Kakao|035720|    Korea|87300.0|     KRW|
|  Com2uS|078340|    Korea|95000.0|     KRW|
+--------+------+---------+-------+--------+



In [19]:
# [+] Temporary View 생성 -> df이 RDD와 마찬가지로 immutable(변경불가) / df 건드리지 못하게 하기위해 Temporary View만듦.
df.createOrReplaceTempView('stocks') # sql처리할 수 있는 view 만들기

In [23]:
# [+] SELECT 문: 하나의 컬럼을 가져오기
ss.sql("SELECT name FROM stocks").show()
# 문자열로 먼저 받아들임 / sql처리 후 새로운 dataframe으로 만들어준것
# SELECT문법 = SELECT 뒤에 변수명(전체 가져오려면 *)과 가져올 곳

+--------+
|    name|
+--------+
|Alphabet|
| Netflix|
|  Amazon|
|   Tesla|
|    HUYA|
| Tencent|
|  Toyota|
| Samsung|
|   Kakao|
|  Com2uS|
+--------+



In [24]:
# [+] SELECT 문: 여러 개의 컬럼을 가져오기
ss.sql("SELECT name, price From stocks").show()

+--------+-------+
|    name|  price|
+--------+-------+
|Alphabet|2282.19|
| Netflix| 190.36|
|  Amazon|2485.63|
|   Tesla| 870.76|
|    HUYA|   4.13|
| Tencent|  377.4|
|  Toyota| 2228.0|
| Samsung|67300.0|
|   Kakao|87300.0|
|  Com2uS|95000.0|
+--------+-------+



In [25]:
# SELECT 문: WHERE 조건을 만족하는 데이터 선택하기
# [+] 한국에 상장된 주식
ss.sql("SELECT name, country from stocks WHERE country='Korea'").show()

+-------+-------+
|   name|country|
+-------+-------+
|Samsung|  Korea|
|  Kakao|  Korea|
| Com2uS|  Korea|
+-------+-------+



In [28]:
# [+] 주가 2000 이상의 주식
ss.sql("SELECT name, price from stocks WHERE price >= 2000").show() # 이건 ''로 안묶어도 ok?

+--------+-------+
|    name|  price|
+--------+-------+
|Alphabet|2282.19|
|  Amazon|2485.63|
|  Toyota| 2228.0|
| Samsung|67300.0|
|   Kakao|87300.0|
|  Com2uS|95000.0|
+--------+-------+



In [33]:
# [+] 주가가 2000 이상인 미국 주식
ss.sql("SELECT name, country FROM stocks WHERE price >= 2000 AND country='USA'").show()
# 앞에 불러오지 않아도 조건 달 수 있는 듯

+--------+-------+
|    name|country|
+--------+-------+
|Alphabet|    USA|
|  Amazon|    USA|
+--------+-------+



In [36]:
# LIKE: 문자열 관련 처리할 때 유용
# [+] 상장국가명이 'U'로 시작하는 주식
ss.sql("SELECT name, country FROM stocks WHERE country LIKE 'U%'").show()
#  %는 U 다음뭐가 나와도 상관 X
# %n하면  맨 마지막 n오는 데이터 나옴. => JAPAN나옴
# %Hong%하면 Hong kong나옴

+--------+-------+
|    name|country|
+--------+-------+
|Alphabet|    USA|
| Netflix|    USA|
|  Amazon|    USA|
|   Tesla|    USA|
|    HUYA|    USA|
+--------+-------+



In [37]:
# NOT LIKE
# [+] 상장국가명이 'U'로 시작하며, 회사명에 'e' 가 들어가지 않는 주식 
ss.sql("SELECT name, country FROM stocks \
        WHERE country LIKE 'U%' AND name NOT LIKE '%e%'").show()

+------+-------+
|  name|country|
+------+-------+
|Amazon|    USA|
|  HUYA|    USA|
+------+-------+



In [39]:
# BETWEEN
# [+] 주가가 1000~10000 사이인 주식
ss.sql("SELECT name, price FROM stocks \
        WHERE price BETWEEN 1000 AND 10000").show()

+--------+-------+
|    name|  price|
+--------+-------+
|Alphabet|2282.19|
|  Amazon|2485.63|
|  Toyota| 2228.0|
+--------+-------+



In [44]:
# 중첩된 SQL 문
# [+] 미국의 주식 중 Tesla 보다 주가가 높은 주식
ss.sql("SELECT name, price FROM stocks \
        WHERE country='USA' \
        AND price > (SELECT price FROM stocks WHERE ticker='TSLA')").show()

# 연산자 우선순위때문에 소괄호로 먼저 묶어주기
# ticker='TSLA'처럼 이름으로 가져오는 것은 위험함. 여러 중복된 결과 얻을 수 있으니까. 정확한 값 가져올 떈 unique한 값으로 불러오는게 좋음


+--------+-------+
|    name|  price|
+--------+-------+
|Alphabet|2282.19|
|  Amazon|2485.63|
+--------+-------+



In [46]:
# ORDER BY
# [+] 주가를 기준으로 오름차순(ASC: Ascending) 정렬
ss.sql("SELECT name, price FROM stocks ORDER BY price ASC").show()

+--------+-------+
|    name|  price|
+--------+-------+
|    HUYA|   4.13|
| Netflix| 190.36|
| Tencent|  377.4|
|   Tesla| 870.76|
|  Toyota| 2228.0|
|Alphabet|2282.19|
|  Amazon|2485.63|
| Samsung|67300.0|
|   Kakao|87300.0|
|  Com2uS|95000.0|
+--------+-------+



In [47]:
# [+] 내림차순(DESC: Descending) 정렬
ss.sql("SELECT name, price FROM stocks ORDER BY price DESC").show()

+--------+-------+
|    name|  price|
+--------+-------+
|  Com2uS|95000.0|
|   Kakao|87300.0|
| Samsung|67300.0|
|  Amazon|2485.63|
|Alphabet|2282.19|
|  Toyota| 2228.0|
|   Tesla| 870.76|
| Tencent|  377.4|
| Netflix| 190.36|
|    HUYA|   4.13|
+--------+-------+



In [48]:
# 집계연산
# [+] 한국 주식의 주가 총합
ss.sql("SELECT sum(price) FROM stocks \
        WHERE country = 'Korea'").show()

+----------+
|sum(price)|
+----------+
|  249600.0|
+----------+



In [49]:
# [+] 한국 주식의 주가 평균
ss.sql("SELECT mean(price) FROM stocks \
        WHERE country = 'Korea'").show()

+-----------+
|mean(price)|
+-----------+
|    83200.0|
+-----------+



In [50]:
# [+] 한국 주식의 개수
ss.sql("SELECT count(price) FROM stocks \
        WHERE country = 'Korea'").show()

+------------+
|count(price)|
+------------+
|           3|
+------------+



In [51]:
# [+] IN => 하나의 조건에서 개별적인 여러 값 중에 하나라도 해당하면 select하는 키워드.
# ex) 통화가 KRW 또는 JPY 주식 가져오기
ss.sql("SELECT name, currency FROM stocks \
        WHERE currency IN ('KRW', 'JPY')").show()

# IN다음 소괄호에 조건들 작성하면 됨

+-------+--------+
|   name|currency|
+-------+--------+
| Toyota|     JPY|
|Samsung|     KRW|
|  Kakao|     KRW|
| Com2uS|     KRW|
+-------+--------+



### 두 개의 데이터프레임을 이용한 SQL 문
데이터베이스나 빅데이터 환경에서는 여러 개의 데이터 테이블들을 결합하여 query를 수행하는 경우가 빈번함
SQL 문의 Join을 이용하여 stocks 와 earnings 데이터프레임을 결합하여 데이터 분석하기

In [53]:
# earnings = EPS(earning per share): 주당 순이익, 전체 순이익을 전체 주식 수로 나눈 값

earnings = [
    ('Alphabet', 110.57, 'USD'), 
    ('Netflix', 10.46, 'USD'),
    ('Amazon', 41.36, 'USD'),
    ('Tesla', 7.4, 'USD'),
    ('HUYA', 0.36, 'USD'),
    ('Tencent', 27.6, 'HKD'),
    ('Toyota', 221.35, 'JPY'),
    ('Samsung', 5777.37, 'KRW'),
    ('Kakao', 3150.95, 'KRW'),
    ('Com2uS', 10852.69, 'KRW')
]

In [54]:
# [+] 스키마 정의에 필요한 클래스 임포트하기
from pyspark.sql.types import StructType, StructField, StringType, FloatType
# StructField는 변수에 대해 정의하는 것. 위에서는 변수 명만 써서 정의 했는데 제대로 하려면 StructField 하는 게 좋음. 
# 어떤 데이터 타입인지 정의해야함. 본 예제에서는 StringType, FloatType이면 됨

In [55]:
# [+] 스키마 정의: 직접 데이터 타입 정하기
earningsSchema = StructType([
    StructField('name', StringType(), True), # 3번째 매개변수는 null 값 허용 -> true
    StructField('eps', FloatType(), True),
    StructField('currency', StringType(), True)
])
# 객체 만들 때 클래스명과 똑같이. => 객체지향 개념

In [56]:
# [+] 데이터프레임 생성
df_earnings = ss.createDataFrame(data=earnings, schema=earningsSchema)

In [58]:
df_earnings.printSchema()
df_earnings.show()

root
 |-- name: string (nullable = true)
 |-- eps: float (nullable = true)
 |-- currency: string (nullable = true)

+--------+--------+--------+
|    name|     eps|currency|
+--------+--------+--------+
|Alphabet|  110.57|     USD|
| Netflix|   10.46|     USD|
|  Amazon|   41.36|     USD|
|   Tesla|     7.4|     USD|
|    HUYA|    0.36|     USD|
| Tencent|    27.6|     HKD|
|  Toyota|  221.35|     JPY|
| Samsung| 5777.37|     KRW|
|   Kakao| 3150.95|     KRW|
|  Com2uS|10852.69|     KRW|
+--------+--------+--------+



In [60]:
# [+] earningDF의 Temporary View 생성
df_earnings.createOrReplaceTempView('earnings')
# 새로 만들거나 이전의 것이 있었으면 대체하겠다

In [61]:
# [+] 함수 형태로 query 수행하기, 예: select()
ss.sql("SELECT * FROM earnings").show()
# df_earnings.select('*').show() => select를 메서드 형태로 처리할 수도 있음 / 참고

+--------+--------+--------+
|    name|     eps|currency|
+--------+--------+--------+
|Alphabet|  110.57|     USD|
| Netflix|   10.46|     USD|
|  Amazon|   41.36|     USD|
|   Tesla|     7.4|     USD|
|    HUYA|    0.36|     USD|
| Tencent|    27.6|     HKD|
|  Toyota|  221.35|     JPY|
| Samsung| 5777.37|     KRW|
|   Kakao| 3150.95|     KRW|
|  Com2uS|10852.69|     KRW|
+--------+--------+--------+



In [63]:
# [+] Inner join 수행 -> stocks기준으로 earnings와 합침
ss.sql("SELECT * FROM stocks JOIN earnings ON stocks.name = earnings.name").show() 
# 어떤 기준으로 JOIN할건지 ON 키워드 뒤에 명시해야함

+--------+------+---------+-------+--------+--------+--------+--------+
|    name|ticker|  country|  price|currency|    name|     eps|currency|
+--------+------+---------+-------+--------+--------+--------+--------+
|Alphabet| GOOGL|      USA|2282.19|     USD|Alphabet|  110.57|     USD|
|  Amazon|  AMZN|      USA|2485.63|     USD|  Amazon|   41.36|     USD|
|  Com2uS|078340|    Korea|95000.0|     KRW|  Com2uS|10852.69|     KRW|
|    HUYA|  HUYA|      USA|   4.13|     USD|    HUYA|    0.36|     USD|
|   Kakao|035720|    Korea|87300.0|     KRW|   Kakao| 3150.95|     KRW|
| Netflix|  NFLX|      USA| 190.36|     USD| Netflix|   10.46|     USD|
| Samsung|005930|    Korea|67300.0|     KRW| Samsung| 5777.37|     KRW|
| Tencent|  0700|Hong Kong|  377.4|     HKD| Tencent|    27.6|     HKD|
|   Tesla|  TSLA|      USA| 870.76|     USD|   Tesla|     7.4|     USD|
|  Toyota|  7203|    Japan| 2228.0|     JPY|  Toyota|  221.35|     JPY|
+--------+------+---------+-------+--------+--------+--------+--

In [66]:
# PER (Price/Earnings Ratio): 주가를 EPS로 나눈 값
# [+] PER 기준으로 가장 저평가된 주식 찾기
# 일반적으로 같은 부류의 종목끼리 비교하는데 위 표에서의 eps는 그게 불가능. 그래서 PER사용
# JOIN연산할 떄는 어떤 df의 변수인지 명시해야함. 그냥 (price / eps)하면 안됨
ss.sql("SELECT stocks.name, (stocks.price / earnings.eps) \
        FROM stocks JOIN earnings ON stocks.name = earnings.name \
        ORDER BY (stocks.price / earnings.eps)").show()

# 10 이하를 저평가된 주가라고 함.

+--------+------------------+
|    name|     (price / eps)|
+--------+------------------+
|  Com2uS|  8.75358977716049|
|  Toyota|10.065506837881282|
|    HUYA|11.472221766357087|
| Samsung|11.648898830245365|
| Tencent|13.673912854486094|
| Netflix| 18.19885270609645|
|Alphabet| 20.64022796688883|
|   Kakao|27.705930386971747|
|  Amazon|60.097436250469926|
|   Tesla|117.67026875379577|
+--------+------------------+



In [ ]:
# 구조화된 데이터 처리할 떄 이렇게 편하게 할 수 있음(RDD로했으면 더 어려웠을 것)